In [3]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from models import NN, Dummy
from utils import add_params, scale_params, subtract_params, average_params, sum_params
import torch

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Experimenting with SGD and adding parameters

In [8]:
model1 = NN(1,1,1)
model2 = NN(1,1,1)

for param in model2.parameters():
    param.data = 2*torch.ones(param.shape)

print('----- model 1 -----')
print('before training')
for param in model1.parameters():
    print(param)

opt = torch.optim.SGD(model1.parameters(), lr=0.01, weight_decay=0.01)
for _ in range(100):
    opt.zero_grad()
    loss = 0
    for param in model1.parameters():
        loss += torch.norm(param)
    loss.backward()
    opt.step()

print('\nafter training')
for param in model1.parameters():
    print(param)

Parameter containing:
tensor([[2.]], requires_grad=True)
Parameter containing:
tensor([2.], requires_grad=True)
Parameter containing:
tensor([[2.]], requires_grad=True)
Parameter containing:
tensor([2.], requires_grad=True)
----- model 1 -----
before training
Parameter containing:
tensor([[0.9322]], requires_grad=True)
Parameter containing:
tensor([0.7053], requires_grad=True)
Parameter containing:
tensor([[-0.0261]], requires_grad=True)
Parameter containing:
tensor([-0.6655], requires_grad=True)

after training
Parameter containing:
tensor([[0.0078]], requires_grad=True)
Parameter containing:
tensor([0.0028], requires_grad=True)
Parameter containing:
tensor([[-0.0061]], requires_grad=True)
Parameter containing:
tensor([-0.0033], requires_grad=True)


### Event-Based ADMM with Torch

In [5]:
# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)*5
lambdas = np.vstack([lam, -lam])
x_init = np.random.randn(lambdas.shape[0],2)*5
delta = 0
rho = 1

x = torch.randn(2,)
agents = [
    EventGlobalConsensusTorch(
        N=len(lambdas), 
        rho=rho, 
        model=Dummy(torch.randn(1,2)),
        loss=torch.norm,
        delta=delta,
        x_init=x, 
        lam_init=Dummy(torch.Tensor(lam)).parameters()
    ) 
    for lam in zip(lambdas)
]

for agent in agents:
    agent.primal_avg = average_params([agent.model.parameters() for agent in agents])
    for param in agent.primal_avg:
        print(param)

for agent in agents:
    for param in agent.primal_avg:
        print(param)

for agent in agents:
    print(agent.delta)

tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
tensor([[ 0.0015, -0.2810]], grad_fn=<DivBackward0>)
0
0
0
0


/var/folders/bv/3kttr09s6dsg653szk2tbhlh0000gn/T/ipykernel_2728/3721919984.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  lam_init=Dummy(torch.Tensor(lam)).parameters()


### Run simulation

In [6]:
from tqdm import tqdm
comm = 0

# print('\nAverage Params')
# for agent in agents:
#     for param in agent.primal_avg:
#         print(param)

t_max = 100
for t in tqdm(range(t_max)):
    
    # Primal Update
    for agent in agents:
        agent.primal_update()

    # Residual update in the case of communication
    sum_of_res = []
    for agent in agents:
        if agent.broadcast: 
            comm += 1
            sum_of_res.append(agent.residual)
    if sum_of_res:
        # print('have residuals')
        res = sum_params(sum_of_res)
        for agent in agents:
            add_params(agent.primal_avg, res)

    # Dual update
    for agent in agents:
        agent.dual_update()

load = comm/(t_max*len(agents))
print(f'Communication load = {comm}')

print('\nParams')
for agent in agents:
    for param in agent.model.parameters():
        print(param)

print('\nAverage')
for agent in agents:
    for param in agent.primal_avg:
        print(param)

  1%|          | 1/100 [00:00<00:08, 11.31it/s]


tensor([[ 0.7597, -1.5201]])
tensor([[-0.8255, -0.6613]])
tensor([[-0.7359, -0.0355]])
tensor([[0.8076, 1.0930]])


TypeError: 'method' object is not iterable